# Initial Setup

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, mixed_precision, regularizers, initializers
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
import seaborn as sns

In [ ]:
# Seeds for reproduciblity
random_seed = 50701

np.random.seed(random_seed)
tf.random.set_seed(random_seed)

tf.keras.utils.set_random_seed(random_seed)
tf.config.experimental.enable_op_determinism()

# Defining scaler and imputer to use throughout. Imputation is temporary so  scaling works properly.
min_max_scaler = MinMaxScaler()
temp_imputer = SimpleImputer(strategy="mean")

# Used when making predictions
le = LabelEncoder()

In [ ]:
# Ensuring using best setup with colab
gpus = tf.config.experimental.list_physical_devices("GPU")
# Flag for if GPU is enabled

if len(gpus) > 0:
    GPU = True
    policy = mixed_precision.Policy("mixed_float16")
    mixed_precision.set_global_policy(policy)

    print("TensorFlow is using the GPU.")
else:
    GPU = False

    print("TensorFlow is not using the GPU.")

TensorFlow is not using the GPU.


# Decide on the Data
These can be modified to test 1 specific model which is broken down fully in the
following sections.

To test all models the section is towards the end under "Train and Evaluate All
Models"

In [ ]:
# Select numerical features for imputation to impute
numerical_features = ["mean arterial pressure", "heart rate", "respiratory rate", "PCO2 (Arterial)",
                      "PO2 (Arterial)", "FiO2", "arterial pH", "sodium", "postassium", "creatinine",
                      "hematocrit", "white blood cell", "HCO3 (serum)"]

# Decide data specifics to test either raw or artificial with different missing mechanisms
is_raw_missing = False  # Artificial otherwise
missing_type = "mcar"  # Artificially missing only - mcar, mnar_central, mnar_upper, mnar_lower

# Depends on missing limits
# - Artificial is a percentage missing i.e. 0.2, 0.5 and 0.7
# - Raw is number of values missing per row i.e. 2, 5, 10
if is_raw_missing:
    level_missing = 5
else:
    level_missing = 0.5

# Read and Prepare Data

In [ ]:
def get_data_and_reference(is_raw_missing=False, level_missing=0.5, missing_type="mcar"):
    """
    Given the specified flags return the specified original data, the numerical features to be
    imputed and a reference specifying the data used.

    :param is_raw_missing: Boolean flag specifying whether the data is artficially missing or raw.
    :param level_missing: A float or integer representing either the percentage of missingness or
                          the number of missing values per row.
    :param missing_type: A string representing the type of missingness used - only applicable to
                         artificially missing data.
    :return:
    """
    if is_raw_missing:
        reference = "raw_{}".format(level_missing)
        df = pd.read_csv("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/missing/raw/measurements_{}.csv".format(level_missing))
    else:
        reference = "artificial_{}_{}".format(level_missing, missing_type)
        df = pd.read_csv("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/missing/artificial/measurements_{}_{}.csv".format(level_missing, missing_type))

    # Shuffling the data
    df = df.sample(frac=1, random_state=507).reset_index(drop=True)

    df_features = df[numerical_features]

    return df, df_features, reference

In [ ]:
def fit_scaler_and_imputer(df_to_fit_to, imputer, scaler):
    """
    Given a dataset containing the training data, the temporary imputer and scaler this will fit
    both the imputer and scale on the given data and return them.

    :param df_to_fit_to:
    :param imputer:
    :param scaler:
    :return:
    """
    # Fitting imputer on original data
    imputer.fit(df_to_fit_to.values)

    # Temporarily imputing the data
    filled_train = imputer.transform(df_to_fit_to.values)

    # Fitting the scaler on the temporary imputation
    scaler.fit(filled_train)

    return imputer, scaler

In [ ]:
def scale_data(features_df, imputer, scaler):
    """
    Given the data with missing features this will temporarily fill them through the given imputer
    and scale the features.

    :param features_df: The dataframe containing the missing features to be scaled.
    :param imputer: The fitteed imputer.
    :param scaler: The fitted scaler.
    :return: The raw data and the scaled features.
    """
    feature_values = features_df.values
    missing_mask = features_df.isna()

    # Using temporary imputation to fill nan's before scalling
    features_temp_filled = imputer.transform(feature_values)

    # Scaling the data
    featured_scaled_filled = scaler.transform(features_temp_filled)

    # Restoring the nans
    featured_scaled_filled[missing_mask] = np.nan

    scaled_features = pd.DataFrame(featured_scaled_filled, index=features_df.index, columns=features_df.columns)

    return scaled_features

In [ ]:
# Get the specified data
shuffled_data, shuffled_features, data_reference = get_data_and_reference(is_raw_missing,
                                                                          level_missing,
                                                                          missing_type)

In [ ]:
# Scale the data, using placeholder imputationss
imputer, scaler = fit_scaler_and_imputer(shuffled_features, temp_imputer, min_max_scaler)
scaled_featured = scale_data(shuffled_features, imputer, scaler)

scaled_featured.head(10)

,mean arterial pressure,heart rate,respiratory rate,PCO2 (Arterial),PO2 (Arterial),FiO2,arterial pH,sodium,postassium,creatinine,hematocrit,white blood cell,HCO3 (serum)
0,0.213720,0.287356,0.085271,NaN,NaN,1.0,0.525641,0.363636,NaN,0.041860,0.453125,0.156395,NaN
1,NaN,0.183908,NaN,0.188976,NaN,1.0,NaN,NaN,NaN,0.037209,0.687500,NaN,NaN
2,NaN,NaN,NaN,0.173228,0.393324,NaN,0.615385,NaN,NaN,NaN,0.398438,0.069583,0.390244
3,0.258575,NaN,0.093023,NaN,0.571843,NaN,NaN,NaN,NaN,0.046512,0.367188,0.088138,0.463415
4,NaN,0.551724,NaN,0.362205,0.044993,1.0,NaN,NaN,0.734177,NaN,NaN,0.237906,NaN
5,0.284960,NaN,0.147287,NaN,NaN,1.0,NaN,0.530303,0.468354,0.069767,0.390625,0.132538,NaN
6,NaN,NaN,0.100775,NaN,0.336720,NaN,NaN,NaN,NaN,NaN,0.406250,NaN,NaN
7,0.224274,NaN,NaN,0.212598,NaN,NaN,NaN,0.469697,0.253165,0.027907,NaN,NaN,0.390244
8,NaN,0.396552,NaN,0.141732,0.119013,NaN,NaN,NaN,NaN,0.027907,0.445312,NaN,NaN
9,0.271768,0.339080,NaN,0.220472,NaN,NaN,NaN,0.439394,NaN,0.027907,NaN,0.059642,0.463415


# Using the Model

## Build the Model

In [ ]:
def build_encoder(n_features, latent_dimensions, l2_reg=1e-5, layer_sizes=None, add_noise=False):
    """
    Used to build an encoder for MIWAE. This takes the temporarily filled dataset and outputs the
    mean and log variance of the latent representation of the variables. The latent representation
    is a compressed representation of the features and can be used to predict the missing values
    when decoded.

    The outputted mean and variance represent the mean/expected value of the features in the latent
    space and the log variance represents the certaintity of the mean (lower is better).

    :param n_features: The number of features that require imputation.
    :param latent_dimensions: The number of dimensions to "compress" the feature data into.
    :param l2_reg: The regulairsation rate applied to the dense hidden layers.
    :param layer_sizes: A tuple representing the sizes and order of layers (e.g. (32, 64, 128))
    :param add_noise: Boolean to add noise to the encoder.
    :return: The specified encoder model, which outputs the mean and log variance.
    """
    # Weight initialiser - need to look into why they used this
    xavier_init = initializers.GlorotUniform()

    # Setting default layers, small to large to increase complexity of captured patterns
    if layer_sizes is None:
        layer_sizes = (32, 64, 128)

    # Input data is the complete dataset with missing values temporarily imputed
    input_layer = layers.Input(shape=(n_features * 2,), name="input_layer")
    x = input_layer

    # Building the architecture from the given layer sizes.
    for size in layer_sizes:
        x = layers.Dense(units=size, kernel_initializer=xavier_init,
                         kernel_regularizer=regularizers.l2(l2_reg))(x)

        if add_noise:
            x = layers.GaussianNoise(0.05)(x)

        x = layers.Activation("relu")(x)

    # Mean of the latent representation of the variables
    mean = layers.Dense(units=latent_dimensions, name="mean")(x)
    # Represents the range/spread of certaintity for the feature variables. Want it to be low,
    # meaning it closer to the mean and the latnet representation is better.
    log_variance = layers.Dense(units=latent_dimensions, name="log_variance")(x)

    # Built model on the given specification, outputting the mean and log variance
    encoder = models.Model(inputs=input_layer, outputs=[mean, log_variance], name="encoder")

    return encoder

In [ ]:
def build_decoder(n_features, latent_dimensions, l2_reg, layer_sizes=None, dropout=0.2):
    """
    Used to build a decoder for the MIWAE. It takes the output of the encoder which is the features
    in their latent space. Using this representation the decoder will "uncompress" the data into
    its original dimensionality, with the missing values imputed.

    :param n_features: The number of features in the data.
    :param latent_dimensions: The number of latent dimensions the data was compressed into.
    :param l2_reg: The regualirsation rate applied to the dense hidden layers.
    :param layer_sizes: Tuple specifying the sizes of the hidden dense layers (e.g. (128, 64, 32))
    :param dropout: The dropout rate applied to the dense layers.
    :return: The specified decoder model, which outputs the imputed dataset.
    """
    # Weight initialiser
    xavier_init = initializers.GlorotUniform()

    # Applying default layer sizs
    if layer_sizes is None:
        layer_sizes = (128, 64, 32)

    # The input of the model is the latent dimensions
    input_layer = layers.Input(shape=(latent_dimensions,), name="latent_input")
    x = input_layer

    # Building the hidden layers. If dropout specified it is added to every layer.
    for size in layer_sizes:
        x = layers.Dense(units=size, kernel_initializer=xavier_init,
                         kernel_regularizer=regularizers.l2(l2_reg))(x)

        if dropout:
            x = layers.Dropout(dropout)(x)

        x = layers.Activation("relu")(x)

    # Reconstructing the original features from the latent space. Using sigmoid as the data is
    # scaled between 0 and 1.
    reconstructed_data = layers.Dense(n_features, activation="sigmoid", name="reconstructed_data")(x)

    # Final decoder model which outputs the reconstructed data with missing values imputed
    decoder = models.Model(inputs=input_layer, outputs=reconstructed_data, name="decoder")

    return decoder

In [ ]:
def find_miwae_loss(original_data, reconstructed_data, missing_mask, latent_mean, latent_log_var,
                    n_latent_samples, latent_samples):
    """
    Finds the MIWAE loss of the reconstructed data, specifically the difference between the real
    observed data and the reconstructed data. Still don't fully understand it and code aided by AI
    because of the lack of understanding.

    :param original_data: The original dataset.
    :param reconstructed_data: The reconstruction of the data from the decoder.
    :param missing_mask: Mask representing where data was originally missing.
    :param latent_mean: The mean outputted from the encoder.
    :param latent_log_var: THe log variance outputted from the encoder.
    :param n_latent_samples: The number of latent samples provided by the encoder.
    :param latent_samples: The samples that have been taken from the encoder.
    :return:
    """
    # Expand the original data and mask to match the reconstructed dimensions. Then repeating the
    # data across the new dimensions.
    original_data = tf.expand_dims(original_data, axis=1)
    original_data = tf.repeat(original_data, n_latent_samples, axis=1)

    missing_mask = tf.expand_dims(missing_mask, axis=1)
    missing_mask = tf.repeat(missing_mask, n_latent_samples, axis=1)

    latent_log_var = tf.clip_by_value(latent_log_var, -10, 10)

    # Remove log from the variance, adding small value to avoid 0 division
    variance = tf.exp(latent_log_var) + 1e-8

    latent_samples = tf.cast(latent_samples, tf.float32)
    latent_mean = tf.cast(latent_mean, tf.float32)
    latent_log_var = tf.cast(latent_log_var, tf.float32)
    variance = tf.cast(variance, tf.float32)

    # Calculate the reconstruction loss of the observed feature values
    feature_recon_loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)(original_data, reconstructed_data)
    recon_loss = -tf.reduce_sum(missing_mask * feature_recon_loss, axis=-1)

    # Expand other dimensions to match the latent dimensions
    latent_mean = tf.expand_dims(latent_mean, axis=1)
    latent_log_var = tf.expand_dims(latent_log_var, axis=1)
    variance = tf.expand_dims(variance, axis=1)

    # Use probability density function (pdf) to determine how likely a sample is to occur in the
    # encoders distribution and the gaussian distribution
    log_likelihood_encoder_distribution = -0.5 * tf.reduce_sum(latent_log_var + tf.square(latent_samples - latent_mean) / variance + tf.math.log(2 * np.pi), axis=-1)

    log_likelihood_normal_distribution = -0.5 * tf.reduce_sum(tf.square(latent_samples) + tf.math.log(2 * np.pi), axis=-1)

    # Find the importance weights
    log_weights = recon_loss + log_likelihood_normal_distribution - log_likelihood_encoder_distribution

    # Normalise the weights
    maximum_weights = tf.reduce_max(log_weights, axis=1, keepdims=True)
    weights = tf.exp(log_weights - maximum_weights)
    weights = weights / tf.reduce_sum(weights, axis=1, keepdims=True)

    # Caclulate the final loss
    weighted_log_likelihood = tf.reduce_sum(weights * log_weights, axis=1)
    miwae_loss = -tf.reduce_mean(weighted_log_likelihood)

    return miwae_loss

## Train the Model

### Training Functions

In [ ]:
def sample_latent(batch_size, latent_mean, latent_log_var, n_latent_samples):
    """
    Given the output of the encoder, sample latent variables from it to be used in reconstruction
    and loss evaluation.

    :param batch_size: The batch size of the current training data/
    :param latent_mean: The latent mean from the encoder.
    :param latent_log_var: The latent log variation from the encoder.
    :param n_latent_samples: The number of samples to get from the latent space.
    :return: The specified number of latent samples.
    """
    # Number of latent variables
    latent_dim = tf.shape(latent_mean)[1]

    # Random gaussian noise
    gaussian_noise = tf.random.normal(shape=(batch_size, n_latent_samples, latent_dim))

    # Standard deviation of the variance
    var_std = tf.exp(0.5 * latent_log_var)

    # Expanding the mean and std to match the latent dimensions
    mean_expanded = tf.expand_dims(latent_mean, axis=1)
    var_std_expanded = tf.expand_dims(var_std, axis=1)

    mean_expanded = tf.cast(mean_expanded, tf.float32)
    gaussian_noise = tf.cast(gaussian_noise, tf.float32)
    var_std_expanded = tf.cast(var_std_expanded, tf.float32)

    # Sampling the latent variables - some maths behind this i dont yet understand
    latent_samples = mean_expanded + gaussian_noise * var_std_expanded

    return latent_samples

In [ ]:
def train_step(batch, encoder, decoder, optimiser, n_latent_samples, training=True):
    """
    Completes one training step for the encoder and decoder, using the provided batch data.

    :param batch: The current batch of data.
    :param encoder: The encoder model.
    :param decoder: The decoder model.
    :param optimiser: The optimiser used for the MIWAE.
    :param n_latent_samples: The number of latent samples to use.
    """
    batch = tf.cast(batch, tf.float32)
    n_latent_samples = tf.cast(n_latent_samples, tf.int32)

    batch_size = tf.shape(batch)[0]

    # Missing mask: 1 if observed, 0 if missing
    missing_mask = tf.where(tf.math.is_nan(batch), tf.zeros_like(batch), tf.ones_like(batch))
    missing_mask = tf.cast(missing_mask, tf.float32)

    # Feature means to impute missing values temporarily
    feature_mean = tf.reduce_sum(tf.where(tf.math.is_nan(batch), tf.zeros_like(batch), batch),
                                 axis=0) / (tf.reduce_sum(missing_mask, axis=0))
    batch_mean_filled = tf.where(tf.math.is_nan(batch), tf.broadcast_to(feature_mean,
                                                                        tf.shape(batch)), batch)

    # Noise for missing values
    noise = tf.random.normal(tf.shape(batch), stddev=1)
    noisy_batch = batch_mean_filled * missing_mask + noise * (1 - missing_mask)

    if training:
        with tf.GradientTape() as tape:
            # Get the latent means and log variances from the encoder
            encoder_input = tf.concat([noisy_batch, missing_mask], axis=1)
            latent_mean, latent_log_var = encoder(encoder_input, training=True)

            # Getting latent samples from the encoders output
            latent_samples = sample_latent(batch_size, latent_mean, latent_log_var, n_latent_samples)

            latent_samples_for_decoder = tf.reshape(latent_samples, (-1, tf.shape(latent_samples)[-1]))

            # Get the rconstruction from the decoder
            reconstructed_data = decoder(latent_samples_for_decoder, training=True)
            reconstructed_data = tf.reshape(reconstructed_data, (batch_size, n_latent_samples, -1))
            reconstructed_data = tf.cast(reconstructed_data, tf.float32)

            # Calculate the MIWAE loss
            loss = find_miwae_loss(original_data=batch, reconstructed_data=reconstructed_data,
                                missing_mask=missing_mask, latent_mean=latent_mean,
                                latent_log_var=latent_log_var, n_latent_samples=n_latent_samples,
                                latent_samples=latent_samples)

            # Find and update the gradients for the encoder an decoder
            gradients = tape.gradient(loss, encoder.trainable_variables + decoder.trainable_variables)
            optimiser.apply_gradients(zip(gradients, encoder.trainable_variables +
                                        decoder.trainable_variables))
    else:
        # validation test - repeating training version but with no gradient updates
        encoder_input = tf.concat([noisy_batch, missing_mask], axis=1)
        latent_mean, latent_log_var = encoder(encoder_input, training=False)

        latent_samples = sample_latent(batch_size, latent_mean, latent_log_var, n_latent_samples)
        latent_samples_for_decoder = tf.reshape(latent_samples, (-1, tf.shape(latent_samples)[-1]))

        reconstructed_data = decoder(latent_samples_for_decoder, training=False)
        reconstructed_data = tf.reshape(reconstructed_data, (batch_size, n_latent_samples, -1))
        reconstructed_data = tf.cast(reconstructed_data, tf.float32)

        loss = find_miwae_loss(original_data=batch, reconstructed_data=reconstructed_data,
                              missing_mask=missing_mask, latent_mean=latent_mean,
                              latent_log_var=latent_log_var, n_latent_samples=n_latent_samples,
                              latent_samples=latent_samples)
    return loss

In [ ]:
def train(training_data, validation_data, n_epochs, encoder, decoder, optimiser,
          n_latent_samples=10, patience=50, progress_bar=True):
    """
    To be written.

    :param training_data: The training data.
    :param validation_data: The validation data.
    :param n_epochs: The max number of epochs to train for.
    :param encoder: The initalised encoder.
    :param decoder: The intialised decoder.
    :param optimiser: The optimiser for the encoder/decoder.
    :param n_latent_samples: The number of latent samples to be used in the loss function.
    :param patience: The limit for how many epochs can be trained without an improvement in epoch
                     loss
    :return: The trained encoder and decoder and their losses.
    """
    # Tracking losses and early stopping
    training_losses = []
    validation_losses = []
    wait = 0
    best_val_loss = np.inf

    # Initialising the best weights to defaults
    best_encoder_weights = encoder.get_weights()
    best_decoder_weights = decoder.get_weights()

    # Using tf.function so it runs more efficiently
    step_function = tf.function(train_step)

    # Training for the specified number of epochs
    for epoch in range(n_epochs):
        train_loss = 0.0
        training_batches = tf.data.experimental.cardinality(training_data).numpy()

        # If set to true then a progress bar is used to show the training progress
        if progress_bar:
            pbar = tqdm(total=training_batches, desc=f"Epoch {epoch + 1}/{n_epochs}", unit="batch",
                        leave=False)
        else:
            pbar = None

        # Training on the batches
        for train_batch in training_data:
            # Train on the batch
            train_step_loss = step_function(train_batch, encoder, decoder, optimiser, n_latent_samples, training=True)

            # Update the overall epoch loss
            train_loss += float(train_step_loss)

            if progress_bar:
                # Update progress bar after each batch
                pbar.update(1)

        # Averaging to get the final epoch loss
        train_loss = train_loss / training_batches
        training_losses.append(train_loss)

        if progress_bar:
            pbar.close()

        # Running validation now, so not training but testing the model
        val_loss = 0
        val_batches = tf.data.experimental.cardinality(validation_data).numpy()

        for val_batch in validation_data:
            step_val_loss = step_function(val_batch, encoder, decoder, optimiser, n_latent_samples, training=False)
            val_loss += float(step_val_loss)

        val_loss /= val_batches
        validation_losses.append(val_loss)

        # if (epoch + 1) % 50 == 0:
        #     tf.print("Epoch {} Train Loss: {} and Validation Loss: {}".format(epoch + 1, train_loss, val_loss))

        # Check if this loss is better than the last epochs
        if val_loss < best_val_loss - 1e-4:
            # Reset wait counter and update the best loss
            wait = 0
            best_val_loss = val_loss

            # Record best weights
            best_encoder_weights = encoder.get_weights()
            best_decoder_weights = decoder.get_weights()

        # Increment wait counter and initiate early stopping if patience exceeded
        else:
            wait += 1

            # If patience exceeded end training and revert models to their best weights
            if wait >= patience:
                print("Early stopping at epoch {}".format(epoch+1))
                encoder.set_weights(best_encoder_weights)
                decoder.set_weights(best_decoder_weights)
                break

    return encoder, decoder, training_losses, validation_losses

In [ ]:
def split_data(features_df, batch_size, validation_split=0.2):
    """
    Given the raw training data this function will scale it, shuffle it and split it into the
    desired batch sizes (with any remainder dropped).

    :param features_df: Dataframe containing the missing features to be imputed.
    :param batch_size: Size of the individual batches for the training data.
    :return: The shuffled and batched data.
    """
    # Shuffling the data and batching it
    dataset = tf.data.Dataset.from_tensor_slices(features_df)
    # Using high buffer size as overshooting should not affect performance
    dataset = dataset.shuffle(buffer_size=len(features_df), seed=507)

    # Splitting into validation and training
    validation_size = int(len(features_df) * validation_split)
    validation_dataset = dataset.take(validation_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    train_dataset = dataset.skip(validation_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return train_dataset, validation_dataset

### Training and Variable Setup

In [ ]:
def initialise_models(n_features, encoder_layers, decoder_layers, latent_dimensions=10, dropout=0.1,
                      l2_reg=1e-5, noise_level=10):
    """
    Given the dimension size, number of features and regularisation rate the models are built and
    returned.

    :n_features: The number of features to be trained on.
    :encoder_layers: The individual layer sizes for the encoder.
    :decoder_layers: The individual layer sizes for the decoder.
    :dropout: The dropout rate for the decoder - check if makes sense.
    :l2_reg: The regularisation rate applied to either model.
    :return: The initialised encoder and decoder.
    """
    # Initialise the models to be trained
    encoder = build_encoder(n_features, latent_dimensions, l2_reg=l2_reg,
                            layer_sizes=encoder_layers)
    decoder = build_decoder(n_features, latent_dimensions, l2_reg=l2_reg,
                            layer_sizes=decoder_layers, dropout=dropout)

    return encoder, decoder

In [ ]:
def initialise_optimisers(learning_rate=1e-5):
    """
    Prepares the optimiser for the MIWAE model with the given learning rate and a schedule with
    exponential decay. Using Adam optimiser and clipping.

    :param learning_rate: The learning rate for the MIWAE encoder and decoder.
    :return: The initialised optimisers for the MIWAE encoder and decoder.
    """
    # Using decay so as training goes on the learning rate will decrease.
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=5000,
    decay_rate=0.98)

    # Define optimiser, using clipnorm to prevent extreme gradients
    optimiser = keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=1.0, beta_1=0.5,
                                      beta_2=0.9)

    return optimiser

### Check Losses

In [ ]:
def plot_losses(miwae_losses, reference=None, show=True):
    """
    Using the losses after training and the reference for saving this will plot the losses over time
    of the training.

    :param miwae_losses: The tracked losses for the MIWAE.
    :param reference: String representing the experiment reference, used to title the plot.
    :param show: Boolean to decide whether plot is shown in environment or saved as always.
    """
    fig = plt.figure()
    plt.plot(range(len(miwae_losses)), miwae_losses, label="MIWAE Loss")
    plt.grid()
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Losses for {}".format(reference))
    plt.legend()

    plt.tight_layout()
    plt.savefig("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Visualisations/losses/{}_losses_separate.png".format(reference))

    if show:
        plt.show()
    else:
        plt.close(fig)

### Saving the Models

In [ ]:
def save_and_clear_models(encoder, decoder, reference):
    """
    This saves the trained models under the given reference and clears the keras sesssion to avoid
    data leakage.

    :param encoder: The trained encoder.
    :param decoder: The trained decoder.
    :param reference: String representing the experiment reference, used to name the model files.
    """
    # Outputting trained models
    print("saving")
    encoder.save("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Models/{}_encoder.keras".format(reference))
    decoder.save("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Models/{}_decoder.keras".format(reference))

    # Clearing state for new model
    keras.backend.clear_session()

# Evaluate the Imputation

## Functions to Test Imputation

In [ ]:
def impute_from_model(missing_features, encoder, decoder, scaler, imputer, n_latent_samples=10):
    """
    Given the data containing missing data and the relevant encoder, decoder, scaler and imputer
    this will return the final imputation from the MIWAE. The scaler and imputer should be fitted
    to the passed missing data.

    The returned data is not complete and requires calling "combine_imputations_with_categorical"
    for demographic data to be added back.

    :param missing_features: Thhe Dataframe containing the missing features to be imputed.
    :param encoder: The trained encoder to impute with.
    :param decoder: The trained decoder to impute with.
    :param scaler: Scaler to scale the data before imputing.
    :param imputer: Imputer for temporary filling before scaling.
    :param n_latent_samples: Number of latent samples to use.
    :return: The imputations from the MIWAE.
    """
    # Getting the feature values and the missing mask
    feature_values = missing_features.values.astype(np.float32)
    missing_mask = (~np.isnan(feature_values)).astype(np.float32)

    # Filling and scaling the features to work with MIWAE
    features_filled = imputer.transform(feature_values)
    features_scaled = scaler.transform(features_filled)

    # Add noise where values are missing - this was filled in by colab so might need verifying
    std = features_scaled.std(axis=0, keepdims=True)
    noise = np.random.normal(0, std, size=features_scaled.shape)
    X_noisy = features_scaled * missing_mask + noise * (1 - missing_mask)

    # Getting the latent mean and log variance from the encoder
    missing_mask = (~np.isnan(feature_values)).astype(np.float32)
    encoder_input = np.concatenate([X_noisy, missing_mask], axis=1)
    latent_mean, latent_log_var = encoder(encoder_input, training=False)

    batch_size = X_noisy.shape[0]

    n_latent_samples = tf.cast(n_latent_samples, tf.int32)

    latent_samples = sample_latent(batch_size=batch_size, latent_mean=latent_mean,
                                   latent_log_var=latent_log_var, n_latent_samples=n_latent_samples)

    # Need to change the dimensions as they wont fit in the decoder currently
    latent_samples_for_decoder = tf.reshape(latent_samples, (batch_size * n_latent_samples,
                                                             latent_samples.shape[2]))

    # Get the rconstruction from the decoder and reshaping into original
    reconstructed_data_shaped = decoder(latent_samples_for_decoder, training=False)
    reconstructed_data = tf.reshape(reconstructed_data_shaped, (batch_size, n_latent_samples, -1))

    # Getting the average of the latent reconstructions
    reconstruction_mean = tf.reduce_mean(reconstructed_data, axis=1).numpy()

    reconstructed_unscaled = scaler.inverse_transform(reconstruction_mean)

    # Replace the missing values with the imputations
    final_imputed_features = np.where(np.isnan(feature_values), reconstructed_unscaled,
                                      feature_values)

    df_pred = pd.DataFrame(final_imputed_features, columns=missing_features.columns,
                           index=missing_features.index)

    return df_pred

In [ ]:
def combine_imputations_with_categorical(imputed_data, missing_data, missing_features):
    """
    Given an imputed dataset containg just the imputed features this function will add back the
    demographic data and return it.

    :param imputed_data: The data which has already been imputed by the MIWAE.
    :param missing_data: The original dataset with the missing values and desired columns.
    :param missing_features: The ?

    :return: The imputed data with the non-feature data returned.
    """
    # Adding the demographic data back
    non_feature_cols = missing_data.drop(columns=missing_features.columns)
    df_final = pd.concat([non_feature_cols, imputed_data], axis=1)

    return df_final

In [ ]:
def plot_imputed_distributions(original_features, imputed_data, reference, show=True):
    """
    This will plot the differences in distributions of the imputed and original datasets. It works
    with both raw and artficially missing data and the histograms are normalised.

    Still work on colours - very bad

    :param original_features: The original dataset with missing values that the imputed data comes from
    :param imputed_data: The imputed dataset
    :param reference:  String representing the experiment reference, used to title the plot.
    :param show: Boolean to decide whether plot is shown in environment or saved as always.
    """
    fig, axes = plt.subplots(4, 4, figsize=(18, 16))
    axes = axes.flatten()

    for i, col in enumerate(original_features):
        ax = axes[i]

        # Density normalises it - look into better colours
        ax.hist(imputed_data[col], alpha=0.5, label="Imputed", color="blue", edgecolor="black",
                density=True)
        ax.hist(original_features[col], alpha=0.5, label="Original", color="orange",
                edgecolor="black", density=True)

        ax.set_title(col)
        ax.tick_params(axis="x")
        ax.tick_params(axis="y")
        ax.legend()

    # Odd number of features so need to remove extra plots
    axes[13].axis('off')
    axes[14].axis('off')
    axes[15].axis('off')

    plt.suptitle("Variable Distribututions for {}".format(reference), fontsize=20)
    plt.tight_layout()
    plt.savefig("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Visualisations/imputed distributions/{}_variable_distribututions.png".format(reference))

    if show:
        plt.show()
    else:
        plt.close(fig)

## Evaluate Artifically Missing

In [ ]:
def evaluate_normalised_mae(ground_truth, imputation, missing_data):
    """
    If the imputed data has a ground truth then this will evaluate the imputations through
    the normalised mean squared error. It will return the normalised MAE per features and the
    average.

    :param ground_truth: The ground truth data with no missing values.
    :param imputation: The imputed dataset from the artificially missing dataset.
    :param missing_data: The original dataset with the missing values.
    :return: The normalised MAE per feature and the average as a DataFrame.
    """
    # Tracking the individual values and the results so they can be averaged
    norm_mae_values = []
    norm_mae_results = {}

    # Identify where the data was missing before imputation
    missing_mask = missing_data.isna()

    for feature in imputation.columns:
        if feature not in missing_mask.columns:
            continue

        # Comparing values that were missing only
        missing_ground_truth = ground_truth[feature][missing_mask[feature]]
        missing_imputed = imputation[feature][missing_mask[feature]]

        # MAE
        feature_mae = mean_absolute_error(missing_ground_truth, missing_imputed)

        # Normalising through IQR
        Q1 = missing_ground_truth.quantile(0.25)
        Q3 = missing_ground_truth.quantile(0.75)
        IQR = Q3 - Q1

        norm_feature_mae = feature_mae / IQR

        norm_mae_results[feature] = norm_feature_mae
        norm_mae_values.append(norm_feature_mae)

    norm_mae_results["average_norm_mae"] = np.mean(norm_mae_values)

    return norm_mae_results

In [ ]:
def plot_normalised_mae(norm_mae_results, reference, show=True):
    """
    This plots the normalised MAE for each of the features

    :param norm_mae_results: The normalised MAE results from the evaluation.
    :param reference: String representing the experiment reference, used to title the plot.
    :param show: Boolean to decide whether plot is shown in environment or saved as always.
    """
    features = list(norm_mae_results.keys())
    values = list(norm_mae_results.values())

    plt.figure(figsize=(10, 5))
    bars = plt.barh(features, values, edgecolor="black")
    plt.xlabel("Normalised Mean Absolute Error")
    plt.title("Normalised Mean Absolute Error by Feature for {}".format(reference))
    plt.gca().invert_yaxis()

    for bar in bars:
      width = bar.get_width()
      plt.text(width + 0.005, bar.get_y() + bar.get_height() / 2, "{:.3f}".format(width))

    plt.tight_layout()
    plt.savefig("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Visualisations/nmae/{}_feature_norm_mae.png".format(reference))

    if show:
        plt.show()
    else:
        plt.close()

In [ ]:
def evaluate_artifically_missing(imputed_data, reference, missing_data, show=True):
    """
    This is a wrapper to read in the ground truth data, evaluate the imputed data through nMAE,
    plot the nMAE per feature and return a dataframe containing the results.
    """
    # Reading in the ground truth data
    reference_df = pd.read_csv("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/missing/raw/measurements_0.csv")

    # Getting the nMAE for the imputations
    norm_mae_results = evaluate_normalised_mae(reference_df, imputed_data, missing_data)

    # Shows the individual nMAE for each feature
    plot_normalised_mae(norm_mae_results, reference, show=show)

    # Final df to represent the evaluation
    results_df = pd.DataFrame(norm_mae_results, index=[0])

    return results_df

# Evaluate Real Missing Through Predictive Performance

In [ ]:
%pip install scikit-optimize

In [ ]:
# Importing optimiser package from scikit
from skopt import gp_minimize, BayesSearchCV
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args

# Used to train model
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

# Tidy memory
import gc

# Used for processing
from types import SimpleNamespace
from datetime import datetime

In [ ]:
# Parameter search space for the prediction model
xgboost_search_space = {
    "gamma": Categorical([0.01, 0.1]),
    "learning_rate": Categorical([0.001, 0.01, 0.1]),
    "max_depth": Categorical([3, 6, 9]),
    "n_estimators": Categorical([100, 200, 300])
}

In [ ]:
def data_setup(score_data):
    """
    Split the data into training and test data for both the features and predicted values. Using stratified sampling to
    get even class distributions.
    :param score_data: The data to be split
    :return: X_train, X_test, y_train, y_test
    """
    # Splitting into features and target variables
    X = score_data[numerical_features].copy()
    y = score_data["outcome_encoded"].copy()

    # Splitting into training and test data, stratifying due to limited data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=507,
                                                        stratify=y)

    return X_train, X_test, y_train, y_test

In [ ]:
def xgb_grid_search_optimisation(score_data, search_reference="no missing data", save_results=True):
    """
    Perform a grid search hyperparameter optimisation for XGBoost using the specified parameters in constants.py.
    Models are evaluated using accuracy, recall and the F-1 score with cross validation.
    :param score_data: The training data as a dataframe, this will be prepared through the defined function prior.
    :param search_reference: Used to label saved results
    """
    # Setting up model for grid search
    xgb_model = xgb.XGBClassifier(enable_categorical=True)
    stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=507)
    X_train, X_test, y_train, y_test = data_setup(score_data)

    bayes_search = BayesSearchCV(estimator=xgb_model, search_spaces=xgboost_search_space,
                                 scoring=["accuracy", "precision", "recall", "f1", "roc_auc"],
                                 refit="roc_auc", n_iter=20, cv=stratified_cv, verbose=0)
    bayes_search.fit(X_train, y_train)

    # Saving results of grid search in order of F1 score
    df = pd.DataFrame(bayes_search.cv_results_)

    # Recording the best results of all grid searches with given reference
    best_result = df.iloc[0].to_frame().T

    return best_result

# Actually Using the MIWAE

In [ ]:
def run_training(data, missing_features, missing_data, is_raw_missing, level_missing, lr,
                 latent_dimensions, batch_size, encoder_layers, decoder_layers, reference, scaler,
                 imputer, n_latent_samples=10, dropout=0, l2_reg=1e-5, show_plots=True,
                 progress_bar=True, save_models=False, timestamp=None):
    """
    This will train the given encoder and decoder with the given hyperparameters for up to 1000
    epochs or until early stopping is triggered by no improvement in either losses.
    """
    # Number of variables to impute (feature length)
    n_features = len(numerical_features)

    # Preparing the models and the optimiser
    encoder, decoder = initialise_models(n_features, encoder_layers, decoder_layers,
                                         latent_dimensions, dropout, l2_reg)
    optimiser = initialise_optimisers(lr)

    # Only keeping n_epochs as testing different batch sizes
    n_epochs = 1000

    training_data, validation_data = split_data(data.values, batch_size)

    # Starting training
    encoder, decoder, train_losses, val_losses = train(training_data, validation_data, n_epochs,
                                                       encoder, decoder, optimiser,
                                                       n_latent_samples, progress_bar=progress_bar)
    if show_plots:
        # Visualising losses for this model
        plot_losses(val_losses, reference=reference, show=show_plots)

    # Using that trained model to impute the data
    complete_imputation = impute_from_model(missing_features, encoder, decoder, scaler, imputer,
                                            n_latent_samples)

    complete_imputation.to_csv("imputed_data.csv")

    # Checking distributions for mean collapse
    plot_imputed_distributions(missing_features, complete_imputation, reference, show=show_plots)

    if "artificial" in reference:
        imputation_scores = evaluate_artifically_missing(complete_imputation, reference,
                                                         missing_data, show=show_plots)
    else:
        # Restoring the outcome column for prediction
        complete_imputation = combine_imputations_with_categorical(complete_imputation,
                                                                   missing_data, missing_features)
        # Encoding outcome for prediction
        complete_imputation["outcome_encoded"] = le.fit_transform(complete_imputation["outcome"])

        # Accuracy, Precision, Recall, F-1 and ROC-AUC with means and std.'s
        imputation_scores = xgb_grid_search_optimisation(complete_imputation,
                                                         search_reference="testing",
                                                         save_results=False)

    if save_models:
        if timestamp is not None:
            reference = reference + "_" + timestamp

        save_and_clear_models(encoder, decoder, reference)

    tf.keras.backend.clear_session()

    # Deleting after their use
    del encoder, decoder, train_losses, val_losses, optimiser, training_data, validation_data, data, complete_imputation
    gc.collect()

    plt.close("all")

    return imputation_scores

# Grid Search

In [ ]:
# Combinations from here will be tested in a bayesian search
miwae_search_space = [
    Categorical([0.01, 0.001, 0.0001, 0.00001], name="lr"),
    Categorical([4, 8, 16, 32], name="latent_dimensions"),
    Categorical([5, 10, 20, 30, 40, 50], name="n_latent_samples"),
    Categorical([0.05, 0.1, 0.2, 0.3], name="dropout"),
    Categorical([1e-5, 1e-4, 1e-3], name="l2_reg"),
    Categorical([32, 64, 128, 256], name="batch_size"),
    Categorical([5, 4, 3, 2, 1], name="encoder_layers_count"),
    Categorical([5, 4, 3, 2, 1], name="decoder_layers_count")
    ]

In [ ]:
# Directory to save the result of each grid search iteration
score_save_path = "/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/results/miwae_{}_individual_scores.csv"

In [ ]:
def generate_layer_sizes(layer_count, model_type="encoder", base_sizes=[128, 64, 32, 16, 8]):
    """
    Given a number of layers this will return the matching layer sizes to be used. i.e. a layer
    count of 3 will return (32, 16, 8), with a maximum of 6 going from 256 to 8.

    :param layer_count: An integer representing the total number of layers to be returned.
    :param model_type: String representing whether the model is a encoder or decoder.
    :param base_sizes: The sizes of the layers to make the selection from, the default is [256, 128,
                       64, 32, 16, 8]
    :return: A list of layer sizes to be used. If decoder it will go from small to large and vice
             versa for the discrminator
    """
    n_sizes = len(base_sizes)

    if layer_count > n_sizes:
        layer_count = n_sizes

    layer_sizes = base_sizes[-layer_count:]

    if model_type == "encoder":
        return layer_sizes
    elif model_type == "decoder":
        # Going from large to small (reverse)
        return layer_sizes[::-1]
    else:
        raise ValueError("model type must be 'encoder' or 'decoder'.")

In [ ]:
def check_previously_run(reference, results_dir):
    """
    If bayesian search or testing has been done for a specific dataset the results can be passed
    to the optimiser to speed up the process.

    :param reference: The reference for the data that is being tested.
    :param results_dir: The directory containing the results of specific grid search.
    :return: Boolean specifying whether the given test has already been completed.
    """
    previous_runs = pd.read_csv(results_dir)

    if "artificial" in reference:
        m_level, m_type = reference.replace("artificial_", "").split("_", 1)
    elif "raw" in reference:
        m_type, m_level = reference.split("_", 1)
    else:
        raise ValueError("Invalid reference provided. Must be either artificial or raw/")

    m_level = float(m_level)

    relevant_row = previous_runs[(previous_runs["missing_type"] == m_type) &
                                (previous_runs["missing_level"] == m_level)]

    if relevant_row.empty:
        return False
    else:
        return True

In [ ]:
def save_scores(scores, reference, timestamp, parameters, save_path):
    """

    :param scores:
    :param reference:
    :param timestamp:
    :param parameters:
    :param save_path:
    """
    row = {
        "reference": reference,
        "timestamp": timestamp,
        "mean_test_accuracy": scores["mean_test_accuracy"][0],
        "std_test_accuracy": scores["std_test_accuracy"][0],
        "mean_test_precision": scores["mean_test_precision"][0],
        "std_test_precision": scores["std_test_precision"][0],
        "mean_test_recall": scores["mean_test_recall"][0],
        "std_test_recall": scores["std_test_recall"][0],
        "mean_test_f1": scores["mean_test_f1"][0],
        "std_test_f1": scores["std_test_f1"][0],
        "mean_test_roc_auc": scores["mean_test_roc_auc"][0],
        "std_test_roc_auc": scores["std_test_roc_auc"][0]
    }

    row.update(parameters)

    if os.path.exists(save_path):
        pd.DataFrame([row]).to_csv(save_path, mode="a", header=False, index=False)
    else:
        pd.DataFrame([row]).to_csv(save_path, mode="w", header=True, index=False)

In [ ]:
def get_previous_search_results(save_path, reference):
    """

    :param save_path:
    :param reference:
    :return:
    """
    if os.path.exists(save_path):
        if "raw" in reference:
            metric = "mean_test_roc_auc"
        elif "artificial" in reference:
            metric = "average_nmae"
        else:
            raise ValueError("Invalid reference provided. Must be either artificial or raw")

        previous_runs = pd.read_csv(save_path)

        x0 = previous_runs[["learning_rate", "latent_dimensions", "n_latent_samples", "dropout", "l2_reg", "batch_size", "encoder_layers_count", "decoder_layers_count"]].values.tolist()
        y0 = previous_runs[metric].values.tolist()

        # ROC-AUC is saved as a positive, but need to minimise in context of bayesian search so
        # making values negative.
        if metric == "mean_test_roc_auc":
            y0 = [-val for val in y0]
    else:
        x0, y0 = None, None

    return x0, y0

In [ ]:
def run_bayesian_optimisation(features, is_raw_missing, missing_features, missing_data,
                              level_missing, reference, grid_search_dir, scaler, imputer):
    """
    Completes a bayesian search with 15 random points and 15 targeted points. This minimises either
    the MAE when using ground truth data or the AUC-ROC when using real missing data.

    :param features: Dataframe containing missing features that require imputation.
    :param is_raw_missing: Boolean flag representing whether using raw missing data or data with
                            ground truth available.
    :param missing_features: The missing features that require imputation.
    :param missing_data: The complete dataset from which the features are selected from.
    :param level_missing: Either the percentage or number of values missing per feature/row.
    :param reference: String representing the reference for this test.
    :param grid_search_dir: String representing the directory to store results in.
    :return: The best parameters and score found in the bayesian search.
    """
    already_tested = check_previously_run(reference, grid_search_dir)

    if already_tested and not is_raw_missing:
        print("Already tested: {}, so skipping.".format(reference))
        return

    features = features.fillna(features.mean())
    save_path = score_save_path.format(reference)

    x0, y0 = get_previous_search_results(save_path, reference)
    # Default is 40 calls
    n_calls = 40
    initial_points = 20

    if x0 == None:
        print("No previous results found for {}, so starting new bayesian search".format(reference))
        x0 = [[0.0001, 8, 10, 0.1, 1e-4, 64, 3, 3]]
    # There are previous runs available to include in search
    else:
        # Checking how many calls remain in total and at random
        n_calls -= len(x0)
        initial_points -= len(x0)

        # There are no remaining calls so returning the best found result
        if n_calls < 1:
            print("Completed all calls so returning best results.")

            best_run_id = y0.index(min(y0))
            best_params = x0[best_run_id]
            best_score = y0[best_run_id]

            # Early return, matching format of the minimise function
            return SimpleNamespace(x=best_params, fun=best_score)

        # Checking if any random calls remain
        elif initial_points < 1:
            print("Completed all random points so now refining.")
            initial_points = 0

        print("Using previous search results for {}, doing {} calls with {} random".format(reference, n_calls, initial_points))

    # Using scikit optimiser function - objective function is to test models using either
    # - nMAE for ground truth data
    # - ROC-AUC for raw data
    @use_named_args(miwae_search_space)
    def objective(lr, latent_dimensions, n_latent_samples, dropout, l2_reg, batch_size,
                  encoder_layers_count, decoder_layers_count):
        # Setting up the model architecutres
        encoder_layers = generate_layer_sizes(encoder_layers_count)
        decoder_layers = generate_layer_sizes(decoder_layers_count)

        params = {
            "learning_rate": lr,
            "latent_dimensions": latent_dimensions,
            "n_latent_samples": n_latent_samples,
            "dropout": dropout,
            "l2_reg": l2_reg,
            "batch_size": batch_size,
            "encoder_layers_count": encoder_layers_count,
            "decoder_layers_count": decoder_layers_count
        }

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Fully training a model and evaluating it given passed values
        scores = run_training(data=features, missing_features=missing_features,
                              missing_data=missing_data, is_raw_missing= is_raw_missing,
                              level_missing=level_missing, lr=lr, timestamp=timestamp,
                              latent_dimensions=latent_dimensions,
                              n_latent_samples=n_latent_samples, dropout=dropout, l2_reg=l2_reg,
                              batch_size=batch_size, encoder_layers=encoder_layers,
                              decoder_layers=decoder_layers, save_models=True, reference=reference,
                              show_plots=False, progress_bar=False, scaler=scaler, imputer=imputer)

        if is_raw_missing:
            # save scores here to a new file with the timestamp
            save_scores(scores, reference, timestamp, params, save_path)
            return -scores["mean_test_roc_auc"].iloc[0]
        else:
            # Returning the average nMAE value
            return scores["average_norm_mae"][0]

    # Minimising nMAE, 20 searches within specified search space. 20 random searches and then 20
    # probability based searches. Can give x0 and y0 as known good configuration to build on
    return gp_minimize(objective, dimensions=miwae_search_space, n_calls=n_calls,
                       n_initial_points=initial_points, random_state=507, verbose=True, x0=x0,
                       y0=y0)

# Train and Evaluate Models on Artifically Missing Data

In [ ]:
# Used to go through all the combinations of artificially missing data
missing_types = ["mcar", "mnar_central", "mnar_upper", "mnar_lower"]
missing_levels = ["0.2", "0.5", "0.7"]

In [ ]:
def grid_search_for_artificial_data():
    """
    Completes a grid search for the artificial data by minimising MAE through a bayesian search,
    tuning the generator and discriminator hyperparameters.
    """
    artificial_grid_search_dir = ("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/artificial_miwae_gridsearch.csv")
    # Ground truth data
    reference_df = pd.read_csv("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/missing/raw/measurements_0.csv")
    imputer, scaler = fit_scaler_and_imputer(reference_df[numerical_features], temp_imputer,
                                             min_max_scaler)
    is_raw_missing = False

    # DF to store the results
    if not os.path.exists(artificial_grid_search_dir):
        blank_df = pd.DataFrame(columns=["missing_type", "missing_level", "best_nmae", "lr",
                                         "latent_dim", "n_latent_samples", "dropout", "l2_reg",
                                         "batch_size", "encoder_layers", "decoder_layers"])
        blank_df.to_csv(artificial_grid_search_dir, index=False)
        # Only required for first appendage
        header = True
    else:
        header = False

    results = []

    # Go through every combination of artificially missing data
    for m_type in missing_types:
        for m_level in missing_levels:
            # Get the data with the specified missing type and level
            artificial_df, artificial_df_missing, artificial_reference = get_data_and_reference(
                is_raw_missing=is_raw_missing, level_missing=m_level, missing_type=m_type)
            # Scaling the data
            artificial_scaled_features = scale_data(artificial_df_missing, imputer, scaler)

            print("Training and testing with {}".format(artificial_reference))

            # Running a bayesian search to optimise
            search_result = run_bayesian_optimisation(features=artificial_scaled_features,
                                                      missing_features=artificial_df_missing,
                                                      missing_data=artificial_df,
                                                      is_raw_missing=is_raw_missing,
                                                      level_missing=m_level,
                                                      reference=artificial_reference,
                                                      grid_search_dir=artificial_grid_search_dir,
                                                      imputer=imputer, scaler=scaler
                                                      )

            if search_result is None:
                continue

            # Getting the optimal parameters found
            lr, latent_dim, n_latent_samples, dropout, l2_reg, batch_size, encoder_layers, \
            decoder_layers = search_result.x

            # Confirming the results
            print("Best nMAE {}".format(search_result.fun))
            print("Best parameters {}".format(search_result.x))

            # Saving results to csv, appending so if runtime expires a recovery is possible
            result_dict = {"missing_type": m_type, "missing_level": m_level,
                           "best_nmae": search_result.fun, "lr": lr, "latent_dim": latent_dim,
                           "n_latent_samples": n_latent_samples, "dropout": dropout,
                           "l2_reg": l2_reg, "batch_size": batch_size,
                           "encoder_layers": encoder_layers, "decoder_layers": decoder_layers
                           }
            result_df = pd.DataFrame([result_dict])
            result_df.to_csv(artificial_grid_search_dir, mode='a', header=header, index=False)

            header=False

    print("Finished")

#### Checking the results on artificial data and finalising the imputations

In [ ]:
artificial_scores = pd.read_csv("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/artificial_miwae_gridsearch.csv")
artificial_scores.head(12)

,missing_type,missing_level,best_nmae,lr,latent_dim,n_latent_samples,dropout,l2_reg,batch_size,encoder_layers,decoder_layers
0,mcar,0.2,0.706812,0.00001,8,10,0.10,0.00001,64,5,2
1,mcar,0.5,0.693397,0.10000,4,10,0.20,0.00001,512,5,4
2,mcar,0.2,0.706812,0.00001,8,10,0.10,0.00001,64,5,2
3,mcar,0.7,0.706394,0.00010,2,10,0.20,0.00001,32,1,4
4,mnar_central,0.2,0.708636,0.01000,4,1,0.30,0.00100,256,3,4
5,mnar_central,0.5,0.736478,0.00001,8,50,0.10,0.00001,64,3,2
6,mnar_central,0.7,0.983048,0.00001,16,20,0.05,0.00010,256,3,3
7,mnar_upper,0.2,0.691473,0.01000,8,1,0.05,0.01000,512,1,3
8,mnar_upper,0.5,0.689939,0.00100,32,5,0.30,0.00010,16,3,1
9,mnar_upper,0.7,0.703584,0.00100,8,1,0.30,0.00100,64,4,2


In [ ]:
def generate_best_imputations(original_tests, test_type="artificial"):
    """
    Function to re-run the best found models with the complete MAE for all the features instead
    of purely the average.

    :param test_type: String representing the main cateogry of missing data (artificial or raw)
    """
    if test_type == "artificial":
        is_raw_missing = False
        best_scores_dir = ("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/results/artificial_miwae_scores.csv")
        df_columns = ["reference", "mean arterial pressure",
                                         "heart rate", "respiratory rate", "PCO2 (Arterial)",
                                         "PO2 (Arterial)", "FiO2", "arterial pH", "sodium",
                                         "postassium", "creatinine", "hematocrit",
                                         "white blood cell", "HCO3 (serum)", "average_norm_mae"]
    elif test_type == "raw":
        is_raw_missing = True
        best_scores_dir = ("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/results/raw_miwae_scores.csv")
        df_columns = ["missing_type", "missing_level", "mean_test_accuracy", "std_test_accuracy",
                      "mean_test_precision", "std_test_precision", "mean_test_recall",
                      "std_test_recall", "mean_test_f1", "std_test_f1", "mean_test_roc_auc",
                      "std_test_roc_auc"]
    else:
        raise ValueError("Invalid score type")

    all_tests = original_tests.groupby(["missing_type", "missing_level"])

    # Check if the results file exists
    if not os.path.exists(best_scores_dir):
        blank_df = pd.DataFrame(columns=df_columns)
        blank_df.to_csv(best_scores_dir, index=False)
        # Only required for first appendage
        header = True
    else:
        header = False

    # Going through each of the grid search results
    for (missing_type, missing_level), group in all_tests:
        # Preparing data and reference for imputation
        shuffled_data, shuffled_features, data_reference = get_data_and_reference(is_raw_missing,
                                                                                  level_missing=missing_level,
                                                                                  missing_type=missing_type)
        tested_combination = check_previously_run(data_reference, best_scores_dir)

        if tested_combination:
            print("Already tested {}, so skipping.".format(data_reference))
            continue
        else:
            print("Testing, {}".format(data_reference))

        imputer, scaler = fit_scaler_and_imputer(shuffled_data[numerical_features], temp_imputer,
                                                 min_max_scaler)

        scaled_features = scale_data(shuffled_features, imputer, scaler)

        # Extracting the best combination of variables
        lr = group["lr"].iloc[0]
        latent_dimensions = group["latent_dim"].iloc[0]
        n_latent_samples = group["n_latent_samples"].iloc[0]
        dropout = group["dropout"].iloc[0]
        l2_reg = group["l2_reg"].iloc[0]
        batch_size = group["batch_size"].iloc[0]
        encoder_layers_count = group["encoder_layers"].iloc[0]
        decoder_layers_count = group["decoder_layers"].iloc[0]

        # Getting the layer sizes for the models
        encoder_layers = generate_layer_sizes(encoder_layers_count)
        decoder_layers = generate_layer_sizes(decoder_layers_count)

        # Training and evaluating the quality of the imputation for the provided model
        scores = run_training(data=scaled_features, missing_features=shuffled_features,
                              missing_data=shuffled_data, is_raw_missing=is_raw_missing,
                              level_missing=missing_level, lr=lr,
                              latent_dimensions=latent_dimensions,
                              n_latent_samples=n_latent_samples, dropout=dropout, l2_reg=l2_reg,
                              batch_size=batch_size, encoder_layers=encoder_layers,
                              decoder_layers=decoder_layers, save_models=True,
                              reference=data_reference, show_plots=False, progress_bar=False,
                              scaler=scaler, imputer=imputer)

        if test_type == "artificial":
            # Preparing scores and saving to shared file
            score_df = pd.DataFrame({
                "missing_type": [missing_type],
                "missing_level": [missing_level],
                "mean arterial pressure": [scores["mean arterial pressure"][0]],
                "heart rate": [scores["heart rate"][0]],
                "respiratory rate": [scores["respiratory rate"][0]],
                "PCO2 (Arterial)": [scores["PCO2 (Arterial)"][0]],
                "PO2 (Arterial)": [scores["PO2 (Arterial)"][0]],
                "FiO2": [scores["FiO2"][0]],
                "arterial pH": [scores["arterial pH"][0]],
                "sodium": [scores["sodium"][0]],
                "postassium": [scores["postassium"][0]],
                "creatinine": [scores["creatinine"][0]],
                "hematocrit": [scores["hematocrit"][0]],
                "white blood cell": [scores["white blood cell"][0]],
                "HCO3 (serum)": [scores["HCO3 (serum)"][0]],
                "average_norm_mae": [scores["average_norm_mae"][0]]
            })
        else:
            score_df = pd.DataFrame({
                "missing_type": [missing_type],
                "missing_level": [missing_level],
                "mean_test_accuracy": [scores["mean_test_accuracy"][0]],
                "std_test_accuracy": [scores["std_test_accuracy"][0]],
                "mean_test_precision": [scores["mean_test_precision"][0]],
                "std_test_precision": [scores["std_test_precision"][0]],
                "mean_test_recall": [scores["mean_test_recall"][0]],
                "std_test_recall": [scores["std_test_recall"][0]],
                "mean_test_f1": [scores["mean_test_f1"][0]],
                "std_test_f1": [scores["std_test_f1"][0]],
                "mean_test_roc_auc": [scores["mean_test_roc_auc"][0]],
                "std_test_roc_auc": [scores["std_test_roc_auc"][0]]
            })

        score_df.to_csv(best_scores_dir, mode='a', header=header, index=False)
        header = False

    complete_scores = pd.read_csv(best_scores_dir)
    complete_scores.head(12)

In [ ]:
generate_best_imputations(artificial_scores, test_type="artificial")

Already tested artificial_0.2_mcar, so skipping.
Already tested artificial_0.5_mcar, so skipping.
Already tested artificial_0.7_mcar, so skipping.
Already tested artificial_0.2_mnar_central, so skipping.
Already tested artificial_0.5_mnar_central, so skipping.
Already tested artificial_0.7_mnar_central, so skipping.
Already tested artificial_0.2_mnar_lower, so skipping.
Already tested artificial_0.5_mnar_lower, so skipping.
Already tested artificial_0.7_mnar_lower, so skipping.
Already tested artificial_0.2_mnar_upper, so skipping.
Already tested artificial_0.5_mnar_upper, so skipping.
Already tested artificial_0.7_mnar_upper, so skipping.


# Train Models on All Levels of Real Missing Data
- evaluation takes place in main code, only getting the imputed datasets to test.

In [ ]:
missing_levels = [2, 5, 10]
is_raw_missing = True

In [ ]:
def grid_search_real_missing():
    """
    Completes a grid search for the artificial data by maximising ROC-AUC through a bayesian search,
    tuning the encoder and decoder hyperparameters.
    """
    real_grid_search_dir = ("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/real_miwae_gridsearch.csv")
    is_raw_missing = True

    # Number of variables to impute
    n_features = len(numerical_features)

    # DF to store the results
    if not os.path.exists(real_grid_search_dir):
        blank_df = pd.DataFrame(columns=["missing_type", "missing_level", "best_roc_auc", "lr",
                                         "latent_dim", "n_latent_samples", "dropout", "l2_reg",
                                         "batch_size", "encoder_layers", "decoder_layers"])
        blank_df.to_csv(real_grid_search_dir, index=False)
        # Only required for first appendage
        header = True
    else:
        header = False

    results = []

    for m_level in missing_levels:
        # Get the data with the specified missing type and level
        real_df, real_df_missing, real_reference = get_data_and_reference(is_raw_missing=is_raw_missing,
                                                                          level_missing=m_level,
                                                                          missing_type=None)

        print("Testing, {}".format(real_reference))

        # Scaling the data
        imputer, scaler = fit_scaler_and_imputer(real_df[numerical_features], temp_imputer,
                                                 min_max_scaler)
        real_scaled_features = scale_data(real_df_missing, imputer, scaler)

        # Running a bayesian search to optimise
        search_result = run_bayesian_optimisation(features=real_scaled_features,
                                                    missing_features=real_df_missing,
                                                    missing_data=real_df,
                                                    is_raw_missing=True,
                                                    level_missing=m_level,
                                                    reference=real_reference,
                                                    grid_search_dir=real_grid_search_dir,
                                                    scaler=scaler, imputer=imputer
                                                    )

        if search_result is None:
            continue

        # Getting the optimal parameters found
        lr, latent_dim, n_latent_samples, dropout, l2_reg, batch_size, encoder_layers, \
         decoder_layers = search_result.x

        # Confirming the results
        print("Best ROC-AUC {}".format(search_result.fun))
        print("Best parameters {}".format(search_result.x))

        # Saving results to csv, appending so if runtime expires a recovery is possible
        result_dict = {"missing_type": "raw", "missing_level": m_level,
                        "best_roc_auc": search_result.fun, "lr": lr, "latent_dim": latent_dim,
                        "n_latent_samples": n_latent_samples, "dropout": dropout, "l2_reg": l2_reg,
                        "batch_size": batch_size, "encoder_layers": encoder_layers,
                        "decoder_layers": decoder_layers
                        }
        result_df = pd.DataFrame([result_dict])
        result_df.to_csv(real_grid_search_dir, mode='a', header=header, index=False)

        header=False

    print("Finished")

In [ ]:
# Training the models to work on the real data with different levels of missingness
grid_search_real_missing()

Testing, raw_2
Already tested: raw_2, so skipping.
None
Testing, raw_5
Already tested: raw_5, so skipping.
None
Testing, raw_10
Already tested: raw_10, so skipping.
None
Finished


In [ ]:
real_scores = pd.read_csv("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/real_miwae_gridsearch.csv")
real_scores.head(12)

,missing_type,missing_level,best_roc_auc,lr,latent_dim,n_latent_samples,dropout,l2_reg,batch_size,encoder_layers,decoder_layers
0,raw,2,-0.846831,0.01,4.0,40.0,0.05,0.0010,32.0,1.0,2.0
1,raw,5,-0.846080,0.01,16.0,40.0,0.05,0.0001,256.0,4.0,1.0
2,raw,10,-0.850313,0.01,8.0,50.0,0.30,0.0010,32.0,2.0,5.0


In [ ]:
def extract_best_scores(test_type="raw"):
    """
    Used to extract the best ROC-AUC scores from the complete grid search. A new csv is created
    containing just the best runs with their hyperparameters and either complete ground truth scores
    or the downstream scores.

    This is used instead of generate best imputations as it is less intensive and more telling.

    :param test_type: String representing the main cateogry of missing data (artificial or raw).
    """
    if test_type == "raw":
        # Only 3 raw files and tested in downstream.
        file_references = ["raw_2", "raw_5", "raw_10"]
        metric = "mean_test_roc_auc"
    elif test_type == "artificial":
        # 12 combinations of missing types and different missing levels
        missing_types = ["mcar", "mnar_central", "mnar_upper", "mnar_lower"]
        missing_levels = ["0.2", "0.5", "0.7"]
        file_references = []

        for missing_type in missing_types:
            for missing_level in missing_levels:
                file_references.append("{}_{}".format(missing_type, missing_level))

        # Tested in their ground truth reconstruction
        metric = "average_norm_mae"
    else:
        raise ValueError("Invalid score type.")
    best_results = []

    # Go through each  of the results files
    for file in file_references:
        file_dir = "/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/results/miwae_{}_individual_scores.csv".format(file)

        complete_scores = pd.read_csv(file_dir)
        complete_scores = complete_scores.drop(columns=["timestamp"])

        # Extract result with the best result, maximising ROC-AUC and minimising nMAE
        if test_type == "raw":
           best_result = complete_scores.iloc[complete_scores[metric].idxmax()]
        else:
            best_result = complete_scores.iloc[complete_scores[metric].idxmin()]
        # Save the best result to list
        best_results.append(best_result)

    # Convert best results to a dataframe and save
    best_results_df = pd.DataFrame(best_results)
    best_results_df.to_csv("/content/drive/MyDrive/Sheffield/6000 Dissertation/Imputing Health Care Data/Data/results/best_{}_miwae_scores.csv".format(test_type), index=False)

    return best_results_df

In [ ]:
extract_best_scores()

,reference,mean_test_accuracy,std_test_accuracy,mean_test_precision,std_test_precision,mean_test_recall,std_test_recall,mean_test_f1,std_test_f1,mean_test_roc_auc,std_test_roc_auc,learning_rate,latent_dimensions,n_latent_samples,dropout,l2_reg,batch_size,encoder_layers_count,decoder_layers_count
29,raw_2,0.878350,0.003003,0.889660,0.004150,0.978017,0.003676,0.931734,0.001558,0.846831,0.008465,0.01,4,40,0.05,0.0010,32,1,2
34,raw_5,0.921607,0.002127,0.929915,0.001612,0.988364,0.001091,0.958249,0.001114,0.846080,0.005431,0.01,16,40,0.05,0.0001,256,4,1
17,raw_10,0.920271,0.001866,0.928430,0.001380,0.988650,0.000964,0.957593,0.000977,0.850313,0.005329,0.01,8,50,0.30,0.0010,32,2,5
